In [58]:
import requests
from lxml import etree
import pandas as pd
from tqdm import tqdm
import time
import random

In [46]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
}

In [51]:
def tags_scrapper(url, headers=headers):
    with requests.request('GET', url, headers=headers) as response:
        html = response.text
        tree = etree.HTML(html)
        # 解析标签
        tag_url = tree.xpath('//div[@class="QuestionHeader-topics"]/div/span/a/@href')
        # 从url中提取标签id
        tag_id = [i.split('/')[-1] for i in tag_url]
        # 解析标签名
        tag_name = tree.xpath('//div[@class="QuestionHeader-topics"]/div/span/a/div/text()')
        res = dict(zip(tag_id, tag_name))
        return res

In [53]:
# 加载数据
top_answers = pd.read_csv('top_answers_for_tags0124.csv')
url_list = top_answers['标题链接'].tolist()
print(len(url_list)) # ==607

607


In [59]:
# 抓取标签
tags_list = []
for url in tqdm(url_list, desc='抓取标签'):
    tags = tags_scrapper(url)
    tags_list.append(tags)
    time.sleep(random.randint(1, 3))

抓取标签: 100%|██████████| 607/607 [25:15<00:00,  2.50s/it]


In [62]:
top_answers['标签'] = tags_list
top_answers.to_excel('question_w_tags0124.xlsx', index=False)